In [26]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image


model = load_model("mnist_model.h5")

def predict_digit(img):
    if img is None:
        return "Lütfen bir rakam çizin!"
    
    try:
        # Sketchpad çıktısı 
        if isinstance(img, dict):
            if 'composite' in img and img['composite'] is not None:
                img = img['composite']
            elif 'image' in img and img['image'] is not None:
                img = img['image']
            else:
                return "Çizim verisi bulunamadı!"
        
        # NumPy array kontrolü
        if not isinstance(img, np.ndarray):
            return "Geçersiz görsel formatı!"
        
        # Eğer RGB ise gri tonlamaya 
        if len(img.shape) == 3:
            img = np.mean(img, axis=2)
        
        # PIL Image'a çevir
        img_pil = Image.fromarray(img.astype('uint8')).convert("L")
        
       
        img_resized = img_pil.resize((28, 28))
        
        # NumPy array'e çevir ve normalize et
        img_array = np.array(img_resized).astype("float32")
        
        # Renkleri ters çevir 
        img_array = 255 - img_array
        
       
        img_array = img_array / 255.0
        
        # Model için reshape 
        img_array = img_array.reshape(1, 28, 28, 1)
        
        # Tahmin 
        prediction = model.predict(img_array, verbose=0)
        predicted_digit = np.argmax(prediction)
        confidence = np.max(prediction) * 100
        
        return f"Modelin tahmini: {predicted_digit} (Güven: %{confidence:.1f})"
        
    except Exception as e:
        return f"Hata: {str(e)}"

# Sketchpad kullanarak çizim arayüzü
try:
    demo = gr.Interface(
        fn=predict_digit,
        inputs=gr.Sketchpad(label="Rakam çizin (0-9)"),
        outputs=gr.Textbox(label="Sonuç"),
        title="🔢 MNIST Rakam Tahmini",
        description="Aşağıdaki alana fare ile 0-9 arası bir rakam çizin."
    )
except:
    # Eski Gradio versiyonu sıkıntısı 
    demo = gr.Interface(
        fn=predict_digit,
        inputs=gr.Image(
            source="canvas" if hasattr(gr.Image, 'source') else None,
            tool="sketch" if hasattr(gr.Image, 'tool') else None,
            type="numpy",
            label="Rakam çizin (0-9)"
        ),
        outputs=gr.Textbox(label="Sonuç"),
        title=" MNIST Rakam Tahmini",
        description="Canvas'a fare ile 0-9 arası bir rakam çizin."
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
